## COM410 - Aprendizado de Máquinas - Semana 7
### Exercício de apoio: Comparando o desempenho de classificadores numa base de dados com faixas de preços de smartphones


O objetivo deste exercício é fazer a comparação do desempenho usando três algoritmos de classificação: k-NN, SVM e Random Forest.
Para isso, use o conjunto de dados sobre faixas de preço de smartphones, contendo várias características dos aparelhos (descritas abaixo).
Verifique os valores obtidos para acurácia, precisão, revocação, medida F1. Use também a validação cruzada para verificar a acurácia dos classificadores.

Mexa nos parâmetros dos algoritmos para conseguir os melhores resultados possíveis. Qual foi o desempenho obtido pelos classificadores? Alguns deles se destacou?




O conjunto de dados contém os atributos descritos a seguir, e está disponível no link abaixo: 

https://raw.githubusercontent.com/higoramario/univesp-com410-aprendizado-de-maquinas/main/mobile-price.csv

#### Classificação de preços de smartphones

- **id**: identificador
- **battery_power**: capacidade de armazenamento (mAh)
- **blue**: tem bluetooth (0 - não, 1 - sim)
- **clock_speed**: velocidade do processador (GHz)
- **dual_sim**: suporte a dois chips SIM (0 - não, 1 - sim)
- **fc**: câmera frontal (mega pixels)
- **four_g**: suporte a 4G (0 - não, 1 - sim)
- **int_memory**: memória interna (GB)
- **m_dep**: profundidade (cm)
- **mobile_wt**: peso (g)
- **n_cores**: núcleos do processador
- **pc**: câmera principal (mega pixela) 
- **px_height**: altura de resolução (pixels)
- **px_width**: largura de resolução (pixels)
- **ram**: memória RAM (MB)
- **sc_h**: altura da tela (cm)
- **sc_w**: largura da tela (cm)
- **talk_time**: tempo de duração da bateria dirante ligações (horas)
- **three_g**: suporte a 3G (0 - não, 1 - sim)
- **touch_screen**: tela sensível a toque  (0 - não, 1 - sim)
- **wifi**: suporte a wifi (0 - não, 1 - sim)

Atributo classe:
- **price_range**: faixa de preço (0 - barato, 1 - médio, 2 - caro, 4 - muito caro)

#### URL original do conjunto de dados
https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification

Fonte: **Mobile Price Classification** | Abhishek Sharma, Kaggle.

#### 1. Importando as bibliotecas **scikit-learn**, **pandas** e **matplotlib**. Importe as bibliotecas.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

plt.rcParams['figure.figsize']=[15,10]
plt.rcParams.update({'font.size': 18})

#### 2.     Importe a base de dados direto da URL e verifique as primeiras linhas. O arquivo contém 2000 registros. 

In [2]:
url = 'https://raw.githubusercontent.com/higoramario/univesp-com410-aprendizado-de-maquinas/main/mobile-price.csv'
telefones = pd.read_csv(url, sep=',')
telefones.head(10)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
5,1859,0,0.5,1,3,0,22,0.7,164,1,...,1004,1654,1067,17,1,10,1,0,0,1
6,1821,0,1.7,0,4,1,10,0.8,139,8,...,381,1018,3220,13,8,18,1,0,1,3
7,1954,0,0.5,1,0,0,24,0.8,187,4,...,512,1149,700,16,3,5,1,1,1,0
8,1445,1,0.5,0,0,0,53,0.7,174,7,...,386,836,1099,17,1,20,1,0,0,0
9,509,1,0.6,1,2,1,9,0.1,93,5,...,1137,1224,513,19,10,12,1,0,0,0


In [3]:
telefones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

#### 3. Separando os atributos dos rótulos e separando os dados de treinamento (80%) e teste (20%)

In [4]:
atributos = telefones.iloc[:,:20]
classes = telefones['price_range']

In [5]:
telefones_treino, telefones_teste, classes_treino, classes_teste = train_test_split(atributos, classes, test_size = 0.2)

#### 4. Diminuindo a escala dos dados para melhorar o treinamento

In [6]:
scaler = StandardScaler()
telefones_treino = scaler.fit_transform(telefones_treino)
telefones_teste = scaler.transform(telefones_teste)

#### 5. Classificando com o algoritmo k-NN

In [7]:
modelo_kNN = KNeighborsClassifier(n_neighbors = 80)
modelo_kNN.fit(telefones_treino,classes_treino)


KNeighborsClassifier(n_neighbors=80)

#### 6. Medindo a acurácia do classificador k-NN

In [8]:
predicao_kNN = modelo_kNN.predict(telefones_teste)
acuracia_kNN = accuracy_score(classes_teste,predicao_kNN)
print('Acurácia de classificação k-NN: {}'.format(round(acuracia_kNN,3)*100)+'%')

Acurácia de classificação k-NN: 67.2%


#### 7. Medindo precisão, revogação e medida F1 do k-NN

In [9]:
print(classification_report(classes_teste, predicao_kNN))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81       107
           1       0.52      0.67      0.59        87
           2       0.61      0.52      0.56       114
           3       0.74      0.73      0.74        92

    accuracy                           0.67       400
   macro avg       0.68      0.68      0.67       400
weighted avg       0.68      0.67      0.67       400



#### 8. Executando o k-NN com validação cruzada

In [10]:
cross_val_score(modelo_kNN, telefones_treino, classes_treino, cv=10)

array([0.64375, 0.6625 , 0.5625 , 0.66875, 0.64375, 0.7    , 0.6375 ,
       0.6375 , 0.625  , 0.6375 ])

#### 9. Classificando com SVM usando a função kernel polinomial

In [11]:
modelo_SVM = SVC(kernel = 'poly', degree = 2, gamma = 'scale', C = 2.0, coef0 = 25)
modelo_SVM.fit(telefones_treino, classes_treino)

SVC(C=2.0, coef0=25, degree=2, kernel='poly')

#### 10. Medindo a acurácia do classificador SVM

In [12]:
predicao_SVM = modelo_SVM.predict(telefones_teste)
acuracia_SVM = accuracy_score(classes_teste,predicao_SVM)
print('Acurácia de classificação SVM: {}'.format(round(acuracia_SVM,3)*100)+'%')

Acurácia de classificação SVM: 93.8%


#### 11. Medindo precisão, revogação e medida F1 do SVM

In [13]:
print(classification_report(classes_teste, predicao_SVM))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       107
           1       0.91      0.93      0.92        87
           2       0.95      0.89      0.92       114
           3       0.93      0.96      0.94        92

    accuracy                           0.94       400
   macro avg       0.94      0.94      0.94       400
weighted avg       0.94      0.94      0.94       400



#### 12. Executando o SVM com validação cruzada

In [14]:
cross_val_score(modelo_SVM, telefones_treino, classes_treino, cv=10)

array([0.95   , 0.96875, 0.98125, 0.95   , 0.95   , 0.96875, 0.9375 ,
       0.95625, 0.95   , 0.9375 ])

#### 13. Classificando com Random Forest

In [15]:
modelo_RF = RandomForestClassifier()
modelo_RF.fit(telefones_treino,classes_treino)

RandomForestClassifier()

#### 14. Medindo a acurácia do classificador Random Forest

In [16]:
predicao_RF = modelo_RF.predict(telefones_teste)
acuracia_RF = accuracy_score(classes_teste,predicao_RF)
print('Acurácia de classificação RF: {}'.format(round(acuracia_RF,3)*100)+'%')

Acurácia de classificação RF: 89.0%


#### 15. Medindo precisão, revogação e medida F1 do Random Forest

In [17]:
print(classification_report(classes_teste, predicao_RF))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94       107
           1       0.77      0.87      0.82        87
           2       0.91      0.81      0.86       114
           3       0.94      0.96      0.95        92

    accuracy                           0.89       400
   macro avg       0.89      0.89      0.89       400
weighted avg       0.89      0.89      0.89       400



#### 16. Executando o Random Forest com validação cruzada

In [18]:
cross_val_score(modelo_RF, telefones_treino,classes_treino, cv=10)

array([0.8625 , 0.8875 , 0.88125, 0.85   , 0.86875, 0.825  , 0.875  ,
       0.83125, 0.86875, 0.86875])

Os resultados dos classificadores podem variar dependendo dos parâmetros que foram selecionados. Neste caderno, SVM teve um desempenho melhor para esses dados, com Random Forest apresentando um desempenho próximo. Já k-NN não funcionou bem com os parâmetros de treinamento que foram usados.

Experimente ver os resultados alterando os parâmetros dos algoritmos para ver se consegue melhores resultados. Se quiser, aplique o conjunto de dados a outros algoritmos de classificação.

**Versões das bibliotecas**

Esse tutorial está usando as seguintes versões de bibliotecas: 

matplotlib==3.2.2

pandas==1.3.5

scikit-learn==1.0.2